In [2]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# OWRI FRAMEWORK

In [3]:
# results save path
base_result_path = '../results/METR-LA/LSTM'
exp_name = 'univariate_AE_weighted_real_time_results.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [4]:
with open('../data/METR-LA/METR_OWRI/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [5]:
# load data of correlated results from pickle file
with open('../results/METR-LA/outlier_scores/AE/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

In [6]:
results['raw'].keys()

dict_keys(['773869', '767541', '767542', '717447', '717446', '717445', '773062', '767620', '737529', '717816', '765604', '767471', '716339', '773906', '765273', '716331', '771667', '716337', '769953', '769402', '769403', '769819', '769405', '716941', '717578', '716960', '717804', '767572', '767573', '773012', '773013', '764424', '769388', '716328', '717819', '769941', '760987', '718204', '718045', '769418', '768066', '772140', '773927', '760024', '774012', '774011', '767609', '769359', '760650', '716956', '769831', '761604', '717495', '716554', '773953', '767470', '716955', '764949', '773954', '767366', '769444', '773939', '774067', '769443', '767750', '767751', '767610', '773880', '764766', '717497', '717490', '717491', '717492', '717493', '765176', '717498', '717499', '765171', '718064', '718066', '765164', '769431', '769430', '717610', '767053', '767621', '772596', '772597', '767350', '767351', '716571', '773023', '767585', '773024', '717483', '718379', '717481', '717480', '717486',

In [7]:
# get target intersections for each trajectory and direction
target_intersections=["717445","773062","737529","769418", "761599", "717587", "717458"]

In [8]:
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[target]['raw']:
        intersection = results[target]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [9]:
def merge_trejectory_data_metr(results):
    data = pd.DataFrame()
    for intersection_name in results['raw']:
        intersection = results['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [10]:
def preprocess_df(df,n_obs, n_features, sequence_length):
    #do scaling:
    scaler = StandardScaler()
    train_portion = 0.8
    test_portion = 0.2
    df_train = df[:math.ceil(len(df)*train_portion)].values
    df_test = df[math.ceil(len(df)*(train_portion)):].values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features]
    test_X, test_y = df_test[:, :n_obs], df_test[:, -n_features]
    scl = scaler.fit(train_X) # fit only on training data
    train_X = scl.transform(train_X)
    test_X = scl.transform(test_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    test_X = test_X.reshape((test_X.shape[0], sequence_length, n_features))
    return train_X, train_y, test_X, test_y, scl

In [11]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [12]:
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device = 'mps',layer_dim=1, dropout_prob = 0.2):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()

        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer

        return out

In [13]:
def train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=100):
    history = {}
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=128, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=128, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        ep_start = time.time()
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]
        history['train_loss'].append(epoch_loss)
    

In [14]:
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=128, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    preds = preds.reshape(-1)
    return preds

In [15]:
# thresholds = [0,0.05,0.1,0.25,0.5]
thresholds = [0.25]
# thresholds = [0,0.25,0.5,0.75,1]
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [16]:
# errors={}
# dfs={}
# intersection_arrays = []
# for trajectory in results.keys():
#     errors[trajectory]={}
#     print("\n \n Starting trajectory: {}".format(trajectory))
#     for direction in results[trajectory]:
#         target = target_intersections[target]
#         errors[target]={}
#         print("Starting direction: {}".format(direction))
#         for threshold in thresholds:
#             errors[target][threshold]={}
#             print("Starting threshold: {}".format(threshold))
#             # ------------------------------------ data processing ---------------------------------------- #
#             data = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
#             ae_score = correlated_results[target] # AE scores of the current trajectory and direction
#             number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
#             if number_of_cols==0: # if threshold is 0, then use the target intersection only
#                 number_of_cols=1
#             top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
#             isct_inc = top_corr_df.index.tolist()
#             df = data[isct_inc].copy(deep=True)
#             df = df[ [target] + [ col for col in df.columns if col != target ] ]  #move target var to front of DF
#             df = df.mul(top_corr_df, axis=1)
#             df = df.astype('float32')
#             sequence_length = 12 # number of time steps to look back
#             n_features = len(isct_inc) # number of features (correlated intersections)
#             output_pred = 1 # number of time steps to predict
#             n_obs = sequence_length * n_features # number of columns in the input
#             reframed = series_to_supervised(df, sequence_length, output_pred)
#             train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)
#             device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# #             # # ------------------------------------ modelling ---------------------------------------------- #
#             # define model, loss function and optimizer
#             model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = 1, dropout_prob= 0.2)
#             model = model.to(device)
#             loss_fn = torch.nn.MSELoss()
#             optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
#             start = time.time()
#             history = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=50)
#             end = time.time()
#             print("Training time: {}".format(end-start))


#             # ------------------------------------ evaluation ---------------------------------------------- #
#             yhat = model_evaluation( model, test_X , device)
#             errors[target][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
#             errors[target][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
#             errors[target][threshold]['history'] = history
#             errors[target][threshold]['df'] = pd.DataFrame({"Real":test_y,"Predicted":yhat})
#             errors[target][threshold]['train_time'] = end-start


# # save errors in save path as pickle file
# with open(results_save_path, 'wb') as handle:
#     pickle.dump(errors, handle)

In [17]:
# ------------------------------------ for metr-la dataset ---------------------------------------- #
errors={}
dfs={}
intersection_arrays = []
for target in target_intersections:
    errors[target]={}
    print("\n \n Starting target: {}".format(target))
    for threshold in thresholds:
        errors[target][threshold]={}
        print("Starting threshold: {}".format(threshold))
        # ------------------------------------ data processing ---------------------------------------- #
        data = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
        ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
        number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
        if number_of_cols==0: # if threshold is 0, then use the target intersection only
            number_of_cols=1
        top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
        isct_inc = top_corr_df.index.tolist()
        df = data[isct_inc].copy(deep=True)
        df = df[ [target] + [ col for col in df.columns if col != target ] ]  #move target var to front of DF
        df = df.mul(top_corr_df, axis=1)
        df = df.astype('float32')
        sequence_length = 12 # number of time steps to look back
        n_features = len(isct_inc) # number of features (correlated intersections)
        output_pred = 1 # number of time steps to predict
        n_obs = sequence_length * n_features # number of columns in the input
        reframed = series_to_supervised(df, sequence_length, output_pred)
        train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)
        
        break
    break


# #             # # ------------------------------------ modelling ---------------------------------------------- #
#         # define model, loss function and optimizer
#         model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = output_pred, dropout_prob= 0.2)
#         model = model.to(device)
#         loss_fn = torch.nn.MSELoss()
#         optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
#         start = time.time()
#         history = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=50)
#         end = time.time()
#         print("Training time: {}".format(end-start))


#         # ------------------------------------ evaluation ---------------------------------------------- #
#         yhat = model_evaluation( model, test_X , device)
#         errors[target][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
#         errors[target][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
#         errors[target][threshold]['history'] = history
#         errors[target][threshold]['df'] = pd.DataFrame({"Real":test_y,"Predicted":yhat})
#         errors[target][threshold]['train_time'] = end-start
#         print("RMSE: {}".format(errors[target][threshold]['RMSE']))


# # save errors in save path as pickle file
# with open(results_save_path, 'wb') as handle:
#     pickle.dump(errors, handle)


 
 Starting target: 717445
Starting threshold: 0.25


In [18]:
reframed

,var1(t-12),var2(t-12),var3(t-12),var4(t-12),var5(t-12),var6(t-12),var7(t-12),var8(t-12),var9(t-12),var10(t-12),...,var43(t),var44(t),var45(t),var46(t),var47(t),var48(t),var49(t),var50(t),var51(t),var52(t)
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-03-01 01:00:00,68.750000,60.686176,65.855652,68.796219,62.223770,54.429718,62.845566,58.252033,64.116989,51.185295,...,64.125404,62.687561,35.220058,64.219978,61.781574,64.809944,55.184044,62.124676,63.208923,61.977329
2012-03-01 01:05:00,68.111115,61.618126,65.773438,65.864639,62.867229,55.934811,61.328743,54.947536,65.727417,51.389874,...,64.044128,57.641602,33.798256,66.508232,62.607494,64.430939,43.854870,53.164646,65.171089,58.446205
2012-03-01 01:10:00,66.250000,58.219261,64.869049,56.097271,62.963062,56.030590,60.508839,60.341244,65.590942,55.113186,...,61.565266,61.921383,36.438747,63.245106,65.559143,64.931763,57.376789,60.053852,60.651337,65.386688
2012-03-01 01:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,59.492771,57.966648,36.723106,60.225697,65.207115,65.513809,49.377335,58.253727,63.438969,61.693214
2012-03-01 01:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,64.978783,62.287060,35.585667,63.245106,61.659718,63.226242,46.778526,56.034019,63.087132,62.829670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-06-27 23:35:00,56.333332,59.370487,64.951271,67.398918,61.607693,50.407005,62.312626,65.707642,65.181511,56.954384,...,61.876816,63.384090,0.000000,65.858315,65.207115,60.749168,57.498608,61.068962,66.145409,63.100254
2012-06-27 23:40:00,54.444443,49.886559,57.551758,57.864441,54.543346,52.213120,61.984665,65.871498,63.216236,49.098606,...,61.565266,62.896523,0.000000,63.488823,64.706146,67.733704,56.524055,60.906544,65.401138,65.021400
2012-06-27 23:45:00,57.222221,55.149319,64.677216,66.522186,61.771980,55.387505,63.187191,64.888344,63.762146,52.153629,...,62.852108,63.709137,0.000000,66.291595,65.532066,65.297234,51.976143,58.362003,63.872002,65.914330


In [21]:
temp = reframed.values

In [32]:
np.concatenate([temp[:,-n_features].reshape(-1,1),temp[:,-n_features*2].reshape(-1,1)], axis=1)

array([[67.5     , 66.28571 ],
       [64.333336, 67.5     ],
       [67.      , 64.333336],
       ...,
       [44.77778 , 50.5     ],
       [53.      , 44.77778 ],
       [49.555557, 53.      ]], dtype=float32)

In [99]:
reframed.iloc[:,624]

timestamp
2012-03-01 01:00:00    67.500000
2012-03-01 01:05:00    64.333336
2012-03-01 01:10:00    67.000000
2012-03-01 01:15:00    68.333336
2012-03-01 01:20:00    65.000000
                         ...    
2012-06-27 23:35:00    54.555557
2012-06-27 23:40:00    50.500000
2012-06-27 23:45:00    44.777779
2012-06-27 23:50:00    53.000000
2012-06-27 23:55:00    49.555557
Name: var1(t), Length: 34260, dtype: float32

In [93]:
reframed.columns[624]

'var1(t)'

In [95]:
len(reframed.columns) 

728

In [73]:
errors['717445'][0.05]['df']

,Real,Predicted
0,49.750000,52.340080
1,50.333332,51.301369
2,54.666668,51.917000
3,54.125000,53.666153
4,55.125000,52.013184
...,...,...
6847,54.555557,53.591049
6848,50.500000,53.520950
6849,44.777779,51.867790
6850,53.000000,51.585758


In [72]:
reframed

,var1(t-12),var2(t-12),var3(t-12),var4(t-12),var5(t-12),var6(t-12),var7(t-12),var8(t-12),var9(t-12),var10(t-12),...,var95(t),var96(t),var97(t),var98(t),var99(t),var100(t),var101(t),var102(t),var103(t),var104(t)
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-03-01 01:00:00,64.125000,64.498360,63.011879,63.984169,65.750557,58.018440,65.945038,67.897125,64.694901,56.355541,...,62.951519,56.532143,65.108170,53.734398,54.536697,64.447777,51.425350,55.619698,46.525066,45.891464
2012-03-01 01:05:00,63.888889,63.980629,65.164040,65.591263,64.104416,54.727196,63.837948,63.506577,60.006863,64.328712,...,63.438900,53.937958,61.135914,46.185806,43.219246,61.071003,43.084328,48.599991,47.338535,45.825924
2012-03-01 01:10:00,65.000000,61.800713,63.502247,65.455070,58.771450,60.099274,64.109833,59.700535,59.436146,55.010834,...,64.137047,63.287556,57.768703,40.478336,51.460873,59.126400,51.780006,35.898911,44.989975,34.566063
2012-03-01 01:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,58.169891,51.514965,63.214111,52.919044,54.470974,64.434639,45.921589,43.456577,45.344227,16.358912
2012-03-01 01:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,64.374153,56.887688,58.123837,49.118450,55.956306,60.781940,49.297405,46.644970,51.602676,44.357815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-06-27 23:35:00,66.000000,65.070587,63.257061,65.046486,67.913673,65.444153,66.883034,67.040764,67.181595,62.101116,...,0.000000,60.048115,63.976997,41.767120,38.487213,62.963047,46.972427,42.616837,63.188019,45.616196
2012-06-27 23:40:00,57.888889,67.032516,56.773350,63.085285,60.948170,65.607353,65.686752,67.530113,62.398441,56.504951,...,0.000000,63.050522,59.426003,49.473522,49.686359,63.974766,42.677128,49.715271,62.230228,46.363358
2012-06-27 23:45:00,66.555557,66.596535,64.183304,63.630062,67.913673,64.628136,65.904251,64.811508,60.767822,65.089363,...,0.000000,59.732075,64.187447,53.550285,52.052380,63.068161,49.389351,53.743401,64.237656,36.073498


In [53]:
df.columns

Index(['717445', '717447', '717452', '765164', '716337', '773023', '773954',
       '772168', '772596', '773974', '717453', '764106', '772597', '717823',
       '764766', '764794', '764781', '773024', '773062', '767542', '718064',
       '769467', '717572', '718066', '717825', '767541', '717608', '717571',
       '773975', '717469', '767351', '716331', '717459', '772178', '717492',
       '767523', '717499', '764120', '772151', '764760', '760024', '717497',
       '716339', '765099', '771667', '765182', '761003', '717463', '765265',
       '773953', '717465', '769346'],
      dtype='object')

In [54]:
top_corr_df.index

Index(['717445', '717447', '717452', '765164', '716337', '773023', '773954',
       '772168', '772596', '773974', '717453', '764106', '772597', '717823',
       '764766', '764794', '764781', '773024', '773062', '767542', '718064',
       '769467', '717572', '718066', '717825', '767541', '717608', '717571',
       '773975', '717469', '767351', '716331', '717459', '772178', '717492',
       '767523', '717499', '764120', '772151', '764760', '760024', '717497',
       '716339', '765099', '771667', '765182', '761003', '717463', '765265',
       '773953', '717465', '769346'],
      dtype='object')

In [55]:
df = df[ [target] + [ col for col in df.columns if col != target ] ]

In [57]:
df.columns

Index(['717445', '717447', '717452', '765164', '716337', '773023', '773954',
       '772168', '772596', '773974', '717453', '764106', '772597', '717823',
       '764766', '764794', '764781', '773024', '773062', '767542', '718064',
       '769467', '717572', '718066', '717825', '767541', '717608', '717571',
       '773975', '717469', '767351', '716331', '717459', '772178', '717492',
       '767523', '717499', '764120', '772151', '764760', '760024', '717497',
       '716339', '765099', '771667', '765182', '761003', '717463', '765265',
       '773953', '717465', '769346'],
      dtype='object')

In [ ]:
AE_results={}
for trajectory in errors.keys():
    for direction in errors[trajectory].keys():
        for threshold in errors[target].keys():
            AE_results[trajectory+'_'+direction+'_'+str(threshold)] = errors[target][threshold]['RMSE']

In [ ]:
AE_results

{'T1_North_0': 10.433202167048048,
 'T1_North_0.25': 10.0138812042315,
 'T1_North_0.5': 9.88758299511825,
 'T1_North_0.75': 9.802343080641764,
 'T1_North_1': 9.744100106086831,
 'T1_South_0': 18.26861793138843,
 'T1_South_0.25': 17.582383164801815,
 'T1_South_0.5': 16.958209562982503,
 'T1_South_0.75': 16.77485566169507,
 'T1_South_1': 16.74054781547215,
 'T2_South_0': 15.412980967347124,
 'T2_South_0.25': 14.563473386526928,
 'T2_South_0.5': 14.810251572331316,
 'T2_South_0.75': 14.759810983331455,
 'T2_South_1': 14.718294367450124,
 'T2_North_0': 15.488593549814883,
 'T2_North_0.25': 15.197303151259735,
 'T2_North_0.5': 14.948777287216922,
 'T2_North_0.75': 15.230001375394924,
 'T2_North_1': 15.014965983684776}